In [ ]:
import pickle
from dask.distributed import Client, Scheduler, SchedulerPlugin

In [ ]:
# Close the previous scheduler and all associated workers if present
try:
    client.shutdown()
    client.close()
except:
    print('Could not shut down old client, was there any to begin with?')

In [ ]:
# Start a client in local cluster mode
client = Client()
print('Dashboard available at', client.dashboard_link)

In [ ]:
# Scheduler plugin that accelerates the
# task graph using dask-accelerated
class SchedulerOptimizer(SchedulerPlugin):

    # Add the scheduler instance to the plugin as an attribute
    # such that we can access the underlying task state
    def __init__(self, scheduler):
        self.scheduler = scheduler

    def update_graph(self, scheduler, dsk=None, keys=None, restrictions=None, **kwargs):
        for key in keys:
            task_state = scheduler.tasks[key]
            scheduler.logg(task_state)
            rs_func = pickle.loads(task_state.run_spec['function'])
            rs_arg = pickle.loads(task_state.run_spec['args'])
            scheduler.logg(rs_func, rs_arg)

In [ ]:
scheduler = client.cluster.scheduler

In [ ]:
# The client dashboard is able to display the info logs
# from the scheduler. Unfortunately, the scheduler does not
# have a method to log data, so we monkey patch the scheduler class
import logging
logger = logging.getLogger("distributed.scheduler")

# The dashboard only exposes info logs, so
# logger.info is used instead of logger.debug
def log_method(self, *msgs):
    logstring = ''
    for msg in msgs:
        logstring += str(msg) + '\t'

    logger.info(logstring)

# The monkey patched method
Scheduler.logg = log_method

# TODO: remove
scheduler.logg('logger test')
scheduler.logg('logger test', 'multiple inputs')

In [ ]:
# Register the plugin with the scheduler
plugin = SchedulerOptimizer(scheduler)
scheduler.add_plugin(plugin)

In [ ]:
# Define a simple function and
# submit a future on the client

# Increment integer values by 1
def inc(x):
    return x + 1

# Y holds the future to the result
y = client.submit(inc, 10)

In [ ]:
# Print the result of the future when it is ready
y.result()

In [ ]:
# Restart the client in case something went wrong
# By default this doesnt run so the entire notebook
# can be executed without restarting the client
if False:
    client.restart()